In [ ]:
import pandas as pd

In [ ]:
train = pd.read_csv('/content/train.csv',encoding='ISO-8859-1')
test = pd.read_csv('/content/test.csv',encoding='ISO-8859-1')

In [ ]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
train_dropped = train.drop(['id','keyword','location'],axis=1)

In [ ]:
test_dropped = test.drop(['id','keyword','location'],axis=1)

In [ ]:
train_X = train_dropped.drop(['target'],axis=1)
train_Y = train_dropped.drop(['text'],axis=1)
test_X = test_dropped

In [ ]:
import tensorflow as tf
import tensorflow_hub as tfhub
import tensorflow_text as tftext

In [ ]:
test_X

,text
0,Just happened a terrible car crash
1,"Heard about #earthquake is different cities, s..."
2,"there is a forest fire at spot pond, geese are..."
3,Apocalypse lighting. #Spokane #wildfires
4,Typhoon Soudelor kills 28 in China and Taiwan
...,...
3258,EARTHQUAKE SAFETY LOS ANGELES ÂÃÃ SAFETY FA...
3259,Storm in RI worse than last hurricane. My city...
3260,Green Line derailment in Chicago http://t.co/U...
3261,MEG issues Hazardous Weather Outlook (HWO) htt...


In [ ]:
bert_preprocess = tfhub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = tfhub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text-layer')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
d_layer = tf.keras.layers.Dropout(0.1, name="dropout-layer")(outputs['pooled_output'])
d_layer = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(d_layer)
model = tf.keras.Model(inputs=[text_input], outputs = [d_layer])

In [ ]:
m= [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=m)

In [15]:
model.fit(train_X, train_Y, epochs=10)

Epoch 1/10
238/238 [==============================] - 3353s 14s/step - loss: 0.6382 - accuracy: 0.6330 - precision: 0.5896 - recall: 0.4797
Epoch 2/10
238/238 [==============================] - 3342s 14s/step - loss: 0.5921 - accuracy: 0.6922 - precision: 0.6638 - recall: 0.5747
Epoch 3/10
238/238 [==============================] - 3337s 14s/step - loss: 0.5784 - accuracy: 0.7045 - precision: 0.6759 - recall: 0.5998
Epoch 4/10
238/238 [==============================] - 3316s 14s/step - loss: 0.5650 - accuracy: 0.7200 - precision: 0.6952 - recall: 0.6200
Epoch 5/10
238/238 [==============================] - 3310s 14s/step - loss: 0.5591 - accuracy: 0.7305 - precision: 0.7094 - recall: 0.6313
Epoch 6/10
238/238 [==============================] - 3318s 14s/step - loss: 0.5501 - accuracy: 0.7341 - precision: 0.7170 - recall: 0.6298
Epoch 7/10
238/238 [==============================] - 3305s 14s/step - loss: 0.5420 - accuracy: 0.7414 - precision: 0.7267 - recall: 0.6380
Epoch 8/10
238/238 [

In [18]:
import numpy as np
y_predicted = model.predict(test_X)
y_predicted = y_predicted.flatten()
y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

array([0, 0, 0, ..., 1, 0, 1])

In [24]:
submission = test.drop(['keyword','location','text'],axis=1)
submission['target']=y_predicted

In [27]:
submission.to_csv("submission.csv", index=False)
submission

,id,target
0,0,0
1,2,0
2,3,0
3,9,1
4,11,1
...,...,...
3258,10861,0
3259,10865,0
3260,10868,1
3261,10874,0
